以下のデータを取得して、`data`ディレクトリに配置。

__New York City Airbnb Open Data__  
  URL:  
  ・https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data/  
  DATA:  
  ・AB_NYC_2019.csv

In [ ]:
%matplotlib inline

import folium
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

### 1. 行、列の指定による抽出

Airbnbの掲載データをロード。

In [ ]:
ab_listings_df = pd.read_csv('data/AB_NYC_2019.csv')

len(ab_listings_df)

データの中身を確認。

In [ ]:
ab_listings_df.head()

#### 1-1. 行番号、列番号の指定
行番号、列番号の指定には`iloc`属性を利用します。  
`iloc`はメソッドではないので、角括弧に番号を指定します。

３行目を抽出。

In [ ]:
ab_listings_df.iloc[2] # 番号は0から始まる。

6列目を抽出。  
行を指定しない場合は「:」を置きます。

In [ ]:
ab_listings_df.iloc[:,5] # 番号は0から始まる。

2〜4行目を抽出。  
「（始まりの番号）:（終わりの番号+1）」と指定。

In [ ]:
ab_listings_df.iloc[1:4] # 番号は0から始まる。

#### 1-2. 行ラベル、列ラベルの指定
行番号、列番号の指定には`loc`属性を利用します。  
`loc`もメソッドではないので、角括弧に番号を指定します。

行ラベルを作るために`neighbourhood`でグループ分けします。

In [ ]:
ab_listings_group_by_neighbourhood_df = ab_listings_df.groupby('neighbourhood').mean().round(2) # とりあえず平均。

ab_listings_group_by_neighbourhood_df.head()

行ラベルで抽出。

In [ ]:
ab_listings_group_by_neighbourhood_df.loc['Arden Heights']

列ラベルで抽出。  
行を指定しない場合は「:」を置きます。

In [ ]:
ab_listings_df.loc[:,'neighbourhood']

列ラベルで抽出する場合は、`loc`を使わずにそのまま角括弧で指定してもOK。

In [ ]:
ab_listings_df['neighbourhood']

### 2. 条件指定による抽出

たとえば「価格が100ドル未満」という判定を`price`にすると、データの行数分のブール値が返ってきます。

In [ ]:
price_under_100_bool = ab_listings_df['price'] < 100

price_under_100_bool.head(10)

これを条件として角括弧に指定すると、Trueに該当する行のみが抽出されます。

In [ ]:
price_under_100_df = ab_listings_df[price_under_100_bool]

price_under_100_df.head()

価格の最大値が100ドル以下であることを確認。

In [ ]:
price_under_100_df['price'].max()

`loc`でも同じ結果が得られます。

In [ ]:
price_under_100_loc_df = ab_listings_df.loc[price_under_100_bool]

price_under_100_loc_df.head()

`query()`を利用することで、SQLのように書くこともできます。

In [ ]:
ab_listings_df.query('host_name == "LisaRoxanne"')

In [ ]:
ab_listings_df.query('365 < minimum_nights < 500')

### 3. ランダム抽出

ランダム抽出は`sample()`を使えば簡単にできます。

説明しやすいよう、毎回同じ結果となるように`random_state`を指定してあります。

In [ ]:
ab_listings_sample_df = ab_listings_df.sample(frac=0.2, random_state=2) # 全体の20%を抽出

len(ab_listings_sample_df) / len(ab_listings_df)

ただし、単純に`sample()`で抽出してしまうと問題がある場合もあります。

たとえばあるホスト（host_id=2787）は複数の物件を掲載していますが、その中の一部しか抽出されません。  
この場合、ホストあたりの物件数に焦点を当てた分析が正しくおこなえません。

In [ ]:
print("host_id=2787の物件数")

print("・抽出前: ", len(ab_listings_df.query('host_id == 2787')), "件")

print("・抽出後: ", len(ab_listings_sample_df.query('host_id == 2787')), "件")

この場合、ホスト単位で抽出をおこないます。  
ただし、抽出件数が若干ずれる可能性があります。

In [ ]:
host_id_sample = pd.Series(ab_listings_df['host_id'].unique()).sample(frac=0.2)

ab_listings_sample_revised_df = ab_listings_df.query('host_id in @host_id_sample') # @マークで変数にアクセス可。

ab_listings_sample_revised_df.head()

データ数がおおよそ20%になっていることを確認。

In [ ]:
len(ab_listings_sample_revised_df) / len(ab_listings_df)